<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>



### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>


### **Notebook Source:**

- This notebook is part of a larger [**NVIDIA Deep Learning Institute**](https://www.nvidia.com/en-us/training/) course titled [**Building RAG Agents with LLMs**](https://www.nvidia.com/en-sg/training/instructor-led-workshops/building-rag-agents-with-llms/). If sharing this material, please give credit and link back to the original course.

<br>

### **Environment Setup:**

In [1]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

NVIDIAEmbeddings.get_available_models(base_url="http://llm_client:9000/v1")
embedder = NVIDIAEmbeddings(
    model="nvidia/embed-qa-4", truncate="END",
    base_url="http://llm_client:9000/v1"
)

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    base_url="http://llm_client:9000/v1"
)

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/docs/guides/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/docs/guides/evaluation/examples/comparisons). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [2]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 64 chunks

Sample Chunk:

Paper: Modeling the impact of Climate transition on real estate prices

Summary: In this work, we propose a model to quantify the impact of the climate
transition on a property in housing market. We begin by noting that property is
an asset in an economy. That economy is organized in sectors, driven by its
productivity which is a multidimensional Ornstein-Uhlenbeck process, while the
climate transition is declined thanks to the carbon price, a continuous
deterministic process. We then extend the sales comparison approach and the
income approach to valuate an energy inefficient real estate asset. We obtain
its value as the difference between the price of an equivalent efficient
building following an exponential Ornstein-Uhlenbeck as well as the actualized
renovation costs and the actualized sum of the future additional energy costs.
These costs are due to the inefficiency of the building, before an optimal
renovation date which depends on the carbon price process. Finally, we carry
out 

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models(base_url="http://llm_client:9000/v1")
embedder = NVIDIAEmbeddings(
    model="nvidia/embed-qa-4", truncate="END",
    base_url="http://llm_client:9000/v1"
)

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    base_url="http://llm_client:9000/v1"
)
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str 
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
generator_chain = chat_prompt | llm 
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)  ## GIVEN

#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

Something interesting from the document "Modeling the impact of Climate transition on real estate prices" by Lionel Sopgoui is that it proposes a model to quantify the impact of climate transition on a property in the housing market. The model takes into account the energy efficiency of a building and computes its value at a given time, considering the actualized sum of cash flows before and after the renovation date, as well as the additional energy costs due to inefficiency. The agent rationally chooses the date of renovation to maximize the value of their property. The model's results align with empirical observations, showing that the order of magnitude of the depreciation obtained is similar to real-world data. This research has practical applications in government policies, asset management, and credit risk, and it opens the door for several extensions, such as modeling the physical risk depending on the location of the dwelling.

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [5]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_template('INSTRUCTION:\n\n{system}\n\nINPUT:\n\n{input}')

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How does the proposed model in the paper determine the impact of climate transition on real estate 
prices, and what specific assumptions are made about the carbon price process?

Answer: The proposed model in the paper extends the sales comparison approach and the income approach to value an 
energy inefficient real estate asset. The value of such an asset is obtained as the difference between the price of
an equivalent efficient building, following an exponential Ornstein-Uhlenbeck process, minus the actualized 
renovation costs and the actualized sum of future additional energy costs due to the building's inefficiency. The 
optimal renovation date, which depends on the carbon price process, is determined using equations (2.16), (2.17), 
and (2.18) in the paper. The carbon price process is assumed to be a continuous deterministic process that 
increases over time, but other scenarios could be considered, such as a carbon price that increases until a certain
year before leveling off or even decreasing. It is also assumed that there is a unique carbon price for the entire 
economy, although the framework can be adapted to various sectors and scenarios.

QA Pair 2

Question: How does the proposed model in the paper calculate the value of a dwelling considering its energy 
efficiency, and which carbon price trajectory scenarios were used for the climate transition simulation?

Answer: The proposed model calculates the value of a dwelling by considering the actualized sum of cash flows 
before, during, and after the renovation date, when the building becomes energy efficient. The optimal renovation 
date is chosen rationally to maximize the property value. Four deterministic carbon price trajectory scenarios were
used for climate transition simulation, namely: Net Zero 2050, which limits global warming to 1.5°C by reaching net
zero CO2 emissions around 2050, with some jurisdictions like the US, EU, and Japan reaching net zero for all GHG by
this point.

QA Pair 3

Question: How does the energy efficiency of a building impact its growth in value over time, considering the 
climate transition and renovation costs, based on simulations of the French housing market?

Answer: According to the presented papers, the energy efficiency of a building impacts its growth in value 
differently over time. While almost all building prices continue to grow with time, those with higher energy 
efficiency are less affected by the impact of the climate transition through energy and renovation costs. 
Furthermore, renovation costs have a stronger impact on building prices in the beginning, and as time passes, the 
effect of renovation costs decreases due to the approaching end of the climate transition and potential renovation 
dates. For example, in the case of Net Zero 2050 scenario, the average annual slowdown in housing price growth 
ranges from -29.776% to -0.263% compared to the Current Policies scenario between 2021 and 2030.

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [7]:
synth_questions

['Question: How does the proposed model in the paper determine the impact of climate transition on real estate prices, and what specific assumptions are made about the carbon price process?',
 'Question: How does the proposed model in the paper calculate the value of a dwelling considering its energy efficiency, and which carbon price trajectory scenarios were used for the climate transition simulation?',
 'Question: How does the energy efficiency of a building impact its growth in value over time, considering the climate transition and renovation costs, based on simulations of the French housing market?']

In [16]:
##  Generate some synthetic answers to the questions above.

rag_answers = []
for i, q in enumerate(synth_questions):
    ## Compute the RAG Answer
    
    rag_answer = ""
    for token in rag_chain.stream(q):
        #print(token, end="")
        rag_answer += token

    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

    rag_answers.append(rag_answer)

QA Pair 1
Question: How does the proposed model in the paper determine the impact of climate transition on real estate 
prices, and what specific assumptions are made about the carbon price process?

RAG Answer: In the paper "Modeling the impact of Climate transition on real estate prices," the authors, Lionel 
Sopgouia and colleagues, propose a model to quantify the impact of climate transition on real estate assets. They 
consider property as an asset in an economy that is organized in sectors, driven by its productivity which is a 
multidimensional Ornstein-Uhlenbeck process. The climate transition is then modeled using the carbon price, which 
is described as a continuous deterministic process.

The impact on real estate prices is determined by extending the sales comparison approach and the income approach. 
The market value of the building serving as collateral to firm n at t >= 0, given the carbon price sequence δ, is 
represented by the equation (2.11) in the paper. This equation takes into account factors such as the source of 
energy p, the renovation costs, and the future additional energy costs due to the building's inefficiency.

The carbon price process is introduced as follows: for 0 <= t≤ < t‡ given, the sequence δ satisfies: for t in [0; 
t≤], δt = δ0, which means the carbon price is constant; for t in (t≤, t‡), δt in (R+)I, the carbon price may 
evolve; and for t >= t‡, δt = δt‡ in (R+)I, which means the carbon price is constant again. It is also assumed that
t → δt is C1(R+, R+).

The authors also provide an example of the carbon price process, where the regulator fixes t≤ >= 0 when the 
transition starts and the transition horizon time t‡ > t≤, the carbon price at the beginning of the transition 
Pcarbon > 0, at the end of the transition δt‡ > Pcarbon, and the annual growth rate ηδ > 0.

Sources:
1. Modeling the impact of Climate transition on real estate prices
2. Published on 2024-08-05
3. Lionel Sopgouia
4. Title: Modeling the impact of Climate transition on real estate prices

QA Pair 2
Question: How does the proposed model in the paper calculate the value of a dwelling considering its energy 
efficiency, and which carbon price trajectory scenarios were used for the climate transition simulation?

RAG Answer: In the paper "Modeling the impact of Climate transition on real estate prices," the proposed model 
calculates the value of a dwelling considering its energy efficiency by obtaining its value as the difference 
between the price of an equivalent efficient building following an exponential Ornstein-Uhlenbeck process, as well 
as the actualized renovation costs and the actualized sum of the future additional energy costs. These costs are 
due to the inefficiency of the building before an optimal renovation date, which depends on the carbon price 
process (Sopgoui, 2024).

Regarding the carbon price trajectory scenarios used for the climate transition simulation, the paper mentions that
the energy price is a deterministic function of the carbon price and the source of energy, and that the climate 
transition is declined thanks to the carbon price, which is a continuous deterministic process. However, the 
specific carbon price trajectory scenarios used for the simulation, such as the Divergent Net Zero or Net Zero 2050
scenarios, are not explicitly mentioned in the provided text excerpts (Sopgoui, 2024). Please refer to the original
paper for more information on the carbon price trajectory scenarios used in the simulation.

Reference(s):
- Sopgoui, Lionel. "Modeling the impact of Climate transition on real estate prices." 2024.

QA Pair 3
Question: How does the energy efficiency of a building impact its growth in value over time, considering the 
climate transition and renovation costs, based on simulations of the French housing market?

RAG Answer: According to the research paper "Modeling the impact of Climate transition on real estate prices" by 
Lionel Sopgoui, energy efficiency significantly impacts a building's growth in value over time, considering the 
climate transition and renovation costs. The study presents a model to quantify the impact of climate transition on
a property in the housing market, with the economy's productivity being a multidimensional Ornstein-Uhlenbeck 
process.

In their approach, the value of an energy-inefficient real estate asset is determined by subtracting the actualized
renovation costs and the actualized sum of future additional energy costs from the price of an equivalent efficient
building. These additional costs stem from the building's inefficiency before an optimal renovation date that 
depends on the carbon price process.

The study considers 5 buildings located in the French economy, each with different energy efficiencies. All 
buildings continue to increase in value over time, but their growths are affected differently by their respective 
energy efficiencies. The impact of energy and renovation costs is more significant in the early stages, and as time
passes, the economy gets closer to the end of the climate transition, as well as potential renovation dates.

Ultimately, the research concludes that the order of magnitude of depreciation due to energy inefficiency, as 
obtained by the model, is consistent with empirical observations. Hence, improving the energy efficiency of a 
building can help minimize the negative impact of climate transition on its growth in value over time.

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access.

In [17]:
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How does the proposed model in the paper determine the impact of climate transition on real 
estate prices, and what specific assumptions are made about the carbon price process?

Synth Answer: Answer: The proposed model in the paper extends the sales comparison approach and the income approach
to value an energy inefficient real estate asset. The value of such an asset is obtained as the difference between 
the price of an equivalent efficient building, following an exponential Ornstein-Uhlenbeck process, minus the 
actualized renovation costs and the actualized sum of future additional energy costs due to the building's 
inefficiency. The optimal renovation date, which depends on the carbon price process, is determined using equations
(2.16), (2.17), and (2.18) in the paper. The carbon price process is assumed to be a continuous deterministic 
process that increases over time, but other scenarios could be considered, such as a carbon price that increases 
until a certain year before leveling off or even decreasing. It is also assumed that there is a unique carbon price
for the entire economy, although the framework can be adapted to various sectors and scenarios.

RAG Answer: In the paper "Modeling the impact of Climate transition on real estate prices," the authors, Lionel 
Sopgouia and colleagues, propose a model to quantify the impact of climate transition on real estate assets. They 
consider property as an asset in an economy that is organized in sectors, driven by its productivity which is a 
multidimensional Ornstein-Uhlenbeck process. The climate transition is then modeled using the carbon price, which 
is described as a continuous deterministic process.

The impact on real estate prices is determined by extending the sales comparison approach and the income approach. 
The market value of the building serving as collateral to firm n at t >= 0, given the carbon price sequence δ, is 
represented by the equation (2.11) in the paper. This equation takes into account factors such as the source of 
energy p, the renovation costs, and the future additional energy costs due to the building's inefficiency.

The carbon price process is introduced as follows: for 0 <= t≤ < t‡ given, the sequence δ satisfies: for t in [0; 
t≤], δt = δ0, which means the carbon price is constant; for t in (t≤, t‡), δt in (R+)I, the carbon price may 
evolve; and for t >= t‡, δt = δt‡ in (R+)I, which means the carbon price is constant again. It is also assumed that
t → δt is C1(R+, R+).

The authors also provide an example of the carbon price process, where the regulator fixes t≤ >= 0 when the 
transition starts and the transition horizon time t‡ > t≤, the carbon price at the beginning of the transition 
Pcarbon > 0, at the end of the transition δt‡ > Pcarbon, and the annual growth rate ηδ > 0.

Sources:
1. Modeling the impact of Climate transition on real estate prices
2. Published on 2024-08-05
3. Lionel Sopgouia
4. Title: Modeling the impact of Climate transition on real estate prices

Synth Evaluation: [1] Justification: While both answers provide detailed explanations of the proposed model, Answer
2 (New Answer) does not discuss the specific assumptions made about the carbon price process as clearly as Answer 1
(Ground Truth). Answer 1 explicitly mentions the assumption that the carbon price process is a continuous 
deterministic process that increases over time and could potentially have other scenarios such as leveling off or 
decreasing. Additionally, Answer 1 highlights the assumption of a unique carbon price for the entire economy, which
is not explicitly stated in Answer 2. Therefore, Answer 1 is more comprehensive and consistent compared to Answer 
2.

Set 2

Question: Question: How does the proposed model in the paper calculate the value of a dwelling considering its 
energy efficiency, and which carbon price trajectory scenarios were used for the climate transition simulation?

...

Synth Evaluation: [1] Justification: While Answer 2 does provide some additional information about the method of 
calculation for the value of a dwelling, it introduces inconsistencies when compared to Answer 1, which is assumed 
to be the ground truth. Answer 2 suggests a different method of calculating the value of a dwelling, which 
contradicts the ground truth provided in Answer 1. Furthermore, Answer 2 does not explicitly mention the four 
deterministic carbon price trajectory scenarios outlined in Answer 1, instead mentioning that specific carbon price
trajectory scenarios used for the simulation are not explicitly mentioned in the provided text excerpts. Therefore,
Answer 2 is inferior to Answer 1 and introduces inconsistencies, making it less preferable from a human 
perspective.

Set 3

Question: Question: How does the energy efficiency of a building impact its growth in value over time, considering 
the climate transition and renovation costs, based on simulations of the French housing market?

Synth Answer: Answer: According to the presented papers, the energy efficiency of a building impacts its growth in 
value differently over time. While almost all building prices continue to grow with time, those with higher energy 
efficiency are less affected by the impact of the climate transition through energy and renovation costs. 
Furthermore, renovation costs have a stronger impact on building prices in the beginning, and as time passes, the 
effect of renovation costs decreases due to the approaching end of the climate transition and potential renovation 
dates. For example, in the case of Net Zero 2050 scenario, the average annual slowdown in housing price growth 
ranges from -29.776% to -0.263% compared to the Current Policies scenario between 2021 and 2030.

RAG Answer: According to the research paper "Modeling the impact of Climate transition on real estate prices" by 
Lionel Sopgoui, energy efficiency significantly impacts a building's growth in value over time, considering the 
climate transition and renovation costs. The study presents a model to quantify the impact of climate transition on
a property in the housing market, with the economy's productivity being a multidimensional Ornstein-Uhlenbeck 
process.

In their approach, the value of an energy-inefficient real estate asset is determined by subtracting the actualized
renovation costs and the actualized sum of future additional energy costs from the price of an equivalent efficient
building. These additional costs stem from the building's inefficiency before an optimal renovation date that 
depends on the carbon price process.

The study considers 5 buildings located in the French economy, each with different energy efficiencies. All 
buildings continue to increase in value over time, but their growths are affected differently by their respective 
energy efficiencies. The impact of energy and renovation costs is more significant in the early stages, and as time
passes, the economy gets closer to the end of the climate transition, as well as potential renovation dates.

Ultimately, the research concludes that the order of magnitude of depreciation due to energy inefficiency, as 
obtained by the model, is consistent with empirical observations. Hence, improving the energy efficiency of a 
building can help minimize the negative impact of climate transition on its growth in value over time.

Synth Evaluation: [2] The second answer is better than the first and does not introduce any inconsistencies.

Justification:
The second answer is more detailed and provides more specific information about the impact of energy efficiency on 
building growth in value over time. It references a research paper by Lionel Sopgoui, providing a clear explanation
of the methodology used in the study. The second answer also gives specific examples of the impact of energy 
efficiency on the value of different buildings and mentions the role of the optimal renovation date depending on 
the carbon price process. This additional information makes the second answer more comprehensive and informative 
than the first.

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [18]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 0.3333333333333333


----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/latest/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/docs/guides/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/docs/modules/agents/concepts) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated recently.**
- **Make sure [`35_langserve.ipynb`](35_langserve.ipynb) is not occupying port 9012 with a running FastAPI service**

**Objective:** On launch, [**`frontend/frontend_block.py`**](frontend/frontend_block.py) had several lines of code which trigger the course pass condition. Once the service was deemed healthy, that sequence of code was replaced with `## secret` by another microservice. Your objective is to invoke that series of commands by using your pipeline to pass the **Evaluation** check! Recall [`35_langserve.ipynb`](35_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

In [20]:
%%js
var url = 'http://'+window.location.host+'/8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

<IPython.core.display.Javascript object>

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NIMs**](https://www.nvidia.com/en-us/ai/), which offers microservice spinup routines that can be deployed on local compute.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.
- [**LangGraph**](https://python.langchain.com/docs/langgraph/) is a framework for graph-based LLM orchestration, and is a natural next step forward for those interested in [multi-agent workflows](https://blog.langchain.dev/langgraph-multi-agent-workflows/).
- [**DSPy**](https://github.com/stanfordnlp/dspy), a flow engineering framework that allows you to optimize LLM orchestration pipelines based on empirical performance results.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>